## WICCI Downscaled Data: Processing and aggregating climate scenario -- Tmin and Tmax for all years -- 20 year intervals -- intermodel std##

Objectives
* aggregate data for tmin and tmax, e.g. mean or standard deviation, for RCP4.5 and RCP8.5
* run for every 20 year model window, e.g. 2021-2040 for 2030 average
* creat new netcdf file(s) for aggregate data for each 20-year timeframe
* calculate intermodel standard deviation differently -- find average of each model within the current time window and then calculate standard deviation across models

Eric Compas, compase@uww.edu 11/17/2021, 1/19/2022, 1/25/2022

In [1]:
import netCDF4
import numpy as np
import os

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
base_folder = "Z:/Climate_Data"
#base_folder = "F:/Climate_Data"
if not os.path.isdir(base_folder):
    print("Base folder not valid")

In [4]:
out_folder = "Z:/Climate_Data/aggregate_files"
#out_folder = "C:/Users/Eric/Dropbox/Climate_Data/aggregate_files"
if not os.path.isdir(base_folder):
    print("Out folder not valid")

The BIG LOOP.

Look across all years, models, and create aggregate file for each year.

In [6]:
# loop across models
models = ["rcp45","rcp85"]
for m in models:
    print("Processing climate scenario "+m)
    
    # get subfolders/GCMs for current model
    GCMs = os.listdir(os.path.join(base_folder,m))
    
    # loop across years in 20-year increments
    for y in range(2030,2091,20):
        print("  Processing base year "+str(y))
        
        # get mean and std for tmin and tmax for current 20-year window
        # loop through GCMS first and then years in current window
        tmin_all_array = []
        tmin_means_array = []
        tmax_all_array = []
        tmax_means_array = []
        for gcm in GCMs:
            print("    Processing GCM "+gcm)
            tmin_gcm_array = []
            tmax_gcm_array = []
            for yr in range(y-9,y+11):
                print("      Processing year "+str(yr))
                realizations = ["01","02","03"]
                for r in realizations:
                    nf = os.path.join(base_folder,m,gcm,"r1i1p1","temp_"+r+"_"+str(yr)+".nc")
                    try:
                        n = netCDF4.Dataset(nf)
                    except:
                        #print("File not found: "+nf)
                        q = 0
                    else:
                        tmin = n.variables['tmin']
                        tmax = n.variables['tmax']
                        # handle leap years -- average Feb 28 and 29
                        if tmin.shape[0] == 366:
                            # get and average Feb 28 and 29 for tmin
                            tmin_leap = tmin[58:60,:,:]
                            tmin_mean_leap = np.ma.mean(tmin_leap,axis=0)
                            tmin_part1 = np.ma.append(tmin[0:58,:,:],np.ma.expand_dims(tmin_mean_leap,axis=0),axis=0)
                            tmin = np.ma.append(tmin_part1,tmin[60:367,:,:],axis=0)
                            # do the same for tmax
                            tmax_leap = tmax[58:60,:,:]
                            tmax_mean_leap = np.ma.mean(tmax_leap,axis=0)
                            tmax_part1 = np.ma.append(tmax[0:58,:,:],np.ma.expand_dims(tmax_mean_leap,axis=0),axis=0)
                            tmax = np.ma.append(tmax_part1,tmax[60:367,:,:],axis=0)
                        else:
                            # get time array for non-leap year (needed for netcdf writing below)
                            # --not optimal coding, but should work--
                            n_time = n.variables['time']
                        
                        # add tmin and tmax array for gcm, year, realization to total array
                        tmin_all_array.append(tmin[:,:,:])
                        tmax_all_array.append(tmax[:,:,:])
                        # add tmin and tmax array for gcm, year, realization to gcm-specific array
                        tmin_gcm_array.append(tmin[:,:,:])
                        tmax_gcm_array.append(tmax[:,:,:])
            
            # calculate mean temperature for gcm over time window (for std calculation)
            tmin_model_mean = np.ma.mean(tmin_gcm_array,axis=0)
            tmax_model_mean = np.ma.mean(tmax_gcm_array,axis=0)
            
            # add this to an array of means (one for each gcm)
            tmin_means_array.append(tmin_model_mean[:,:,:])
            tmax_means_array.append(tmax_model_mean[:,:,:])
            
            
        # calculate mean from total array    
        tmin_mean = np.ma.mean(tmin_all_array,axis=0)
        tmax_mean = np.ma.mean(tmax_all_array,axis=0)
        print("Final tmin_mean shape:")
        print(np.shape(tmin_mean))
        
        # calculate std from gcm means array
        tmin_std = np.ma.std(tmin_means_array,axis=0)
        tmax_std = np.ma.std(tmax_means_array,axis=0)
        print("Final tmin_std shape:")
        print(np.shape(tmin_std))
        
        # write netcdf files with results
        filename = "temp_"+m+"_"+str(y)+"_20yr.nc"
        newfile = os.path.join(out_folder,filename)
        ncfile = netCDF4.Dataset(newfile,mode='w',format='NETCDF4_CLASSIC')
        lat_dim = ncfile.createDimension('lat', 48)     # latitude axis
        lon_dim = ncfile.createDimension('lon', 62)    # longitude axis
        time_dim = ncfile.createDimension('time', None) # unlimited axis (can be appended to).
        
        ncfile.title='Aggregate tmin and tmax values for WICCI downscaled climate data for all GCMs for '+m+' and 20-year window around year '+str(y)
        ncfile.subtitle="Data source: UW-Madison WICCI, University of Wisconsin Probabilistic Downscaling (UWPD) dataset; Data aggregation: UW-Whitewater GIS Center"
        lat = ncfile.createVariable('lat', np.float64, ('lat',))
        lat.units = 'degrees_north'
        lat.long_name = 'latitude'
        lon = ncfile.createVariable('lon', np.float64, ('lon',))
        lon.units = 'degrees_east'
        lon.long_name = 'longitude'
        time = ncfile.createVariable('time', np.float64, ('time',))
        time.units = 'days since '+str(y)+'-01-01'
        time.long_name = 'time'
        
        temp_tmin_mean = ncfile.createVariable('tmin_mean',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        temp_tmin_mean.units = 'C' # degrees Celius
        temp_tmin_mean.standard_name = 'mean of minimum daily temperature across 20-year window'
        temp_tmin_mean.missing_value = -32768

        temp_tmin_std = ncfile.createVariable('tmin_std',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        temp_tmin_std.units = 'C' # degrees Celius
        temp_tmin_std.standard_name = 'standard deviation of minimum daily temperature across 20-year window'
        temp_tmin_std.missing_value = -32768

        temp_tmax_mean = ncfile.createVariable('tmax_mean',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        temp_tmax_mean.units = 'C' # degrees Celius
        temp_tmax_mean.standard_name = 'mean of maximum daily temperature across 20-year window'
        temp_tmax_mean.missing_value = -32768

        temp_tmax_std = ncfile.createVariable('tmax_std',np.float32,('time','lat','lon')) # note: unlimited dimension is leftmost
        temp_tmax_std.units = 'C' # degrees Celius
        temp_tmax_std.standard_name = 'standard deviation of maximum daily temperature across 20-year window'
        temp_tmax_std.missing_value = -32768
        
        # Write latitudes, longitudes
        # Note: the ":" is necessary in these "write" statements
        n_lat = n.variables['lat']
        n_lon = n.variables['lon']
        lat[:] = n_lat[:]
        lon[:] = n_lon[:]
        
        # write temp variable
        temp_tmin_mean[:,:,:] = tmin_mean  # Appends data along unlimited dimension
        temp_tmin_std[:,:,:] = tmin_std
        temp_tmax_mean[:,:,:] = tmax_mean  # Appends data along unlimited dimension
        temp_tmax_std[:,:,:] = tmax_std
        
        # write time
        
        time[:] = n_time[:]
        print("Final n_time shape:")
        print(np.shape(n_time))
        
        # close file
        ncfile.close()
        
        # report progress
        print("Wrote: "+filename)

Processing climate scenario rcp45
  Processing base year 2030
    Processing GCM ACCESS1-0
      Processing year 2021
      Processing year 2022
      Processing year 2023
      Processing year 2024
      Processing year 2025
      Processing year 2026
      Processing year 2027
      Processing year 2028
      Processing year 2029
      Processing year 2030
      Processing year 2031
      Processing year 2032
      Processing year 2033
      Processing year 2034
      Processing year 2035
      Processing year 2036
      Processing year 2037
      Processing year 2038
      Processing year 2039
      Processing year 2040
    Processing GCM ACCESS1-3
      Processing year 2021
      Processing year 2022
      Processing year 2023
      Processing year 2024
      Processing year 2025
      Processing year 2026
      Processing year 2027
      Processing year 2028
      Processing year 2029
      Processing year 2030
      Processing year 2031
      Processing year 2032
      Processing

      Processing year 2026
      Processing year 2027
      Processing year 2028
      Processing year 2029
      Processing year 2030
      Processing year 2031
      Processing year 2032
      Processing year 2033
      Processing year 2034
      Processing year 2035
      Processing year 2036
      Processing year 2037
      Processing year 2038
      Processing year 2039
      Processing year 2040
    Processing GCM MIROC-ESM
      Processing year 2021
      Processing year 2022
      Processing year 2023
      Processing year 2024
      Processing year 2025
      Processing year 2026
      Processing year 2027
      Processing year 2028
      Processing year 2029
      Processing year 2030
      Processing year 2031
      Processing year 2032
      Processing year 2033
      Processing year 2034
      Processing year 2035
      Processing year 2036
      Processing year 2037
      Processing year 2038
      Processing year 2039
      Processing year 2040
    Processing GCM MIROC-E

      Processing year 2049
      Processing year 2050
      Processing year 2051
      Processing year 2052
      Processing year 2053
      Processing year 2054
      Processing year 2055
      Processing year 2056
      Processing year 2057
      Processing year 2058
      Processing year 2059
      Processing year 2060
    Processing GCM GFDL-CM3
      Processing year 2041
      Processing year 2042
      Processing year 2043
      Processing year 2044
      Processing year 2045
      Processing year 2046
      Processing year 2047
      Processing year 2048
      Processing year 2049
      Processing year 2050
      Processing year 2051
      Processing year 2052
      Processing year 2053
      Processing year 2054
      Processing year 2055
      Processing year 2056
      Processing year 2057
      Processing year 2058
      Processing year 2059
      Processing year 2060
    Processing GCM GFDL-ESM2G
      Processing year 2041
      Processing year 2042
      Processing year 20

      Processing year 2057
      Processing year 2058
      Processing year 2059
      Processing year 2060
    Processing GCM NorESM1-M
      Processing year 2041
      Processing year 2042
      Processing year 2043
      Processing year 2044
      Processing year 2045
      Processing year 2046
      Processing year 2047
      Processing year 2048
      Processing year 2049
      Processing year 2050
      Processing year 2051
      Processing year 2052
      Processing year 2053
      Processing year 2054
      Processing year 2055
      Processing year 2056
      Processing year 2057
      Processing year 2058
      Processing year 2059
      Processing year 2060
Final tmin_mean shape:
(365, 48, 62)
Final tmin_std shape:
(365, 48, 62)
Final n_time shape:
(365,)
Wrote: temp_rcp45_2050_20yr.nc
  Processing base year 2070
    Processing GCM ACCESS1-0
      Processing year 2061
      Processing year 2062
      Processing year 2063
      Processing year 2064
      Processing year 2065


      Processing year 2080
    Processing GCM IPSL-CM5A-MR
      Processing year 2061
      Processing year 2062
      Processing year 2063
      Processing year 2064
      Processing year 2065
      Processing year 2066
      Processing year 2067
      Processing year 2068
      Processing year 2069
      Processing year 2070
      Processing year 2071
      Processing year 2072
      Processing year 2073
      Processing year 2074
      Processing year 2075
      Processing year 2076
      Processing year 2077
      Processing year 2078
      Processing year 2079
      Processing year 2080
    Processing GCM IPSL-CM5B-LR
      Processing year 2061
      Processing year 2062
      Processing year 2063
      Processing year 2064
      Processing year 2065
      Processing year 2066
      Processing year 2067
      Processing year 2068
      Processing year 2069
      Processing year 2070
      Processing year 2071
      Processing year 2072
      Processing year 2073
      Processing y

...

      Processing year 2055
      Processing year 2056
      Processing year 2057
      Processing year 2058
      Processing year 2059
      Processing year 2060
    Processing GCM CanESM2
      Processing year 2041
      Processing year 2042
      Processing year 2043
      Processing year 2044
      Processing year 2045
      Processing year 2046
      Processing year 2047
      Processing year 2048
      Processing year 2049
      Processing year 2050
      Processing year 2051
      Processing year 2052
      Processing year 2053
      Processing year 2054
      Processing year 2055
      Processing year 2056
      Processing year 2057
      Processing year 2058
      Processing year 2059
      Processing year 2060
    Processing GCM CMCC-CESM
      Processing year 2041
      Processing year 2042
      Processing year 2043
      Processing year 2044
      Processing year 2045
      Processing year 2046
      Processing year 2047
      Processing year 2048
      Processing year 2049

      Processing year 2043
      Processing year 2044
      Processing year 2045
      Processing year 2046
      Processing year 2047
      Processing year 2048
      Processing year 2049
      Processing year 2050
      Processing year 2051
      Processing year 2052
      Processing year 2053
      Processing year 2054
      Processing year 2055
      Processing year 2056
      Processing year 2057
      Processing year 2058
      Processing year 2059
      Processing year 2060
    Processing GCM MIROC-ESM-CHEM
      Processing year 2041
      Processing year 2042
      Processing year 2043
      Processing year 2044
      Processing year 2045
      Processing year 2046
      Processing year 2047
      Processing year 2048
      Processing year 2049
      Processing year 2050
      Processing year 2051
      Processing year 2052
      Processing year 2053
      Processing year 2054
      Processing year 2055
      Processing year 2056
      Processing year 2057
      Processing year

      Processing year 2066
      Processing year 2067
      Processing year 2068
      Processing year 2069
      Processing year 2070
      Processing year 2071
      Processing year 2072
      Processing year 2073
      Processing year 2074
      Processing year 2075
      Processing year 2076
      Processing year 2077
      Processing year 2078
      Processing year 2079
      Processing year 2080
    Processing GCM CSIRO-Mk3-6-0
      Processing year 2061
      Processing year 2062
      Processing year 2063
      Processing year 2064
      Processing year 2065
      Processing year 2066
      Processing year 2067
      Processing year 2068
      Processing year 2069
      Processing year 2070
      Processing year 2071
      Processing year 2072
      Processing year 2073
      Processing year 2074
      Processing year 2075
      Processing year 2076
      Processing year 2077
      Processing year 2078
      Processing year 2079
      Processing year 2080
    Processing GCM GFD

      Processing year 2074
      Processing year 2075
      Processing year 2076
      Processing year 2077
      Processing year 2078
      Processing year 2079
      Processing year 2080
    Processing GCM MRI-CGCM3
      Processing year 2061
      Processing year 2062
      Processing year 2063
      Processing year 2064
      Processing year 2065
      Processing year 2066
      Processing year 2067
      Processing year 2068
      Processing year 2069
      Processing year 2070
      Processing year 2071
      Processing year 2072
      Processing year 2073
      Processing year 2074
      Processing year 2075
      Processing year 2076
      Processing year 2077
      Processing year 2078
      Processing year 2079
      Processing year 2080
    Processing GCM MRI-ESM1
      Processing year 2061
      Processing year 2062
      Processing year 2063
      Processing year 2064
      Processing year 2065
      Processing year 2066
      Processing year 2067
      Processing year 206

      Processing year 2097
      Processing year 2098
      Processing year 2099
      Processing year 2100
    Processing GCM HadGEM2-CC
      Processing year 2081
      Processing year 2082
      Processing year 2083
      Processing year 2084
      Processing year 2085
      Processing year 2086
      Processing year 2087
      Processing year 2088
      Processing year 2089
      Processing year 2090
      Processing year 2091
      Processing year 2092
      Processing year 2093
      Processing year 2094
      Processing year 2095
      Processing year 2096
      Processing year 2097
      Processing year 2098
      Processing year 2099
      Processing year 2100
    Processing GCM inmcm4
      Processing year 2081
      Processing year 2082
      Processing year 2083
      Processing year 2084
      Processing year 2085
      Processing year 2086
      Processing year 2087
      Processing year 2088
      Processing year 2089
      Processing year 2090
      Processing year 2091